0️⃣ Notebook Goal (Markdown)

Simulate how an executive team, under time pressure, might misinterpret an engagement drop using surface-level metrics and correlations — leading to a potentially incorrect product decision.

In [2]:
import pandas as pd

users = pd.read_csv("data_design/users.csv", parse_dates=["signup_date"])
events = pd.read_csv("data_design/events.csv", parse_dates=["event_time", "first_seen"])
features = pd.read_csv("data_design/features.csv", parse_dates=["launch_date"])


In [3]:
feature_launch = features.loc[0, "launch_date"]

events["date"] = events["event_time"].dt.date

pre_period = events[events["event_time"] < feature_launch]
post_period = events[events["event_time"] >= feature_launch]


In [4]:
def compute_esau(df):
    daily_sessions = df.groupby("date").size()
    daily_active_users = df.groupby("date")["user_id"].nunique()
    return (daily_sessions / daily_active_users).mean()

esau_pre = compute_esau(pre_period)
esau_post = compute_esau(post_period)

esau_pre, esau_post


(np.float64(1.7880730748520337), np.float64(1.7621798510177635))

In [5]:
exposed = events[events["feature_exposed"] == 1]
not_exposed = events[events["feature_exposed"] == 0]

esau_exposed = compute_esau(exposed)
esau_not_exposed = compute_esau(not_exposed)

esau_exposed, esau_not_exposed


(np.float64(1.4165956168637042), np.float64(1.5887408561783822))

In [6]:
from scipy.stats import ttest_ind


In [7]:
def daily_esau(df):
    daily_sessions = df.groupby("date").size()
    daily_active_users = df.groupby("date")["user_id"].nunique()
    return (daily_sessions / daily_active_users)

esau_daily_pre = daily_esau(pre_period)
esau_daily_post = daily_esau(post_period)


In [8]:
t_stat_pre_post, p_value_pre_post = ttest_ind(
    esau_daily_pre,
    esau_daily_post,
    equal_var=False
)

t_stat_pre_post, p_value_pre_post


(np.float64(8.767128678100304), np.float64(1.2367011538515507e-13))

In [9]:
esau_exposed_daily = daily_esau(exposed)
esau_not_exposed_daily = daily_esau(not_exposed)


In [10]:
t_stat_exp, p_value_exp = ttest_ind(
    esau_exposed_daily,
    esau_not_exposed_daily,
    equal_var=False
)

t_stat_exp, p_value_exp


(np.float64(-6.603619428073771), np.float64(2.6623591137221277e-09))

In [11]:
events["post"] = (events["event_time"] >= feature_launch).astype(int)
events["treated"] = events["feature_exposed"]


In [12]:
daily_group_metrics = (
    events
    .groupby(["date", "treated", "post"])
    .agg(
        sessions=("user_id", "count"),
        active_users=("user_id", "nunique")
    )
    .reset_index()
)

daily_group_metrics["esau"] = (
    daily_group_metrics["sessions"] /
    daily_group_metrics["active_users"]
)


In [14]:
pre_treated = daily_group_metrics.query("treated == 1 and post == 0")["esau"].mean()
post_treated = daily_group_metrics.query("treated == 1 and post == 1")["esau"].mean()

pre_control = daily_group_metrics.query("treated == 0 and post == 0")["esau"].mean()
post_control = daily_group_metrics.query("treated == 0 and post == 1")["esau"].mean()

did_estimate = (post_treated - pre_treated) - (post_control - pre_control)

pre_treated, post_treated, pre_control, post_control, did_estimate


(nan,
 np.float64(1.4165956168637042),
 np.float64(1.7880730748520337),
 np.float64(1.2842055220936364),
 np.float64(nan))

In [15]:
import statsmodels.formula.api as smf

did_model = smf.ols(
    "esau ~ treated + post + treated:post",
    data=daily_group_metrics
).fit()

did_model.summary().tables[1]


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,1.7881,0.002,961.140,0.000,1.784,1.792
treated,0.0662,0.002,40.711,0.000,0.063,0.069
post,-0.5039,0.003,-170.353,0.000,-0.510,-0.498
treated:post,0.0662,0.002,40.711,0.000,0.063,0.069
